In [27]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
from sklearn import neighbors
from sklearn import linear_model

NA_THRESH = 4

In [28]:
import numpy as np
import csv
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

data = pd.read_csv('movie_metadata.csv')
to_drop = ['director_name', 'num_critic_for_reviews', 'actor_2_name','actor_1_name', 'movie_title', 'num_voted_users'
           , 'actor_3_name', 'plot_keywords', 'movie_imdb_link', 'num_user_for_reviews']

# str_list = []

# for name, value in data.iteritems():
#     if type(value[1]) == str:
#          str_list.append(name)
            
features_list = data.columns.difference(to_drop)
movie_data = data[features_list]
#print(np.sum(movie_num.isnull()))

print(movie_data)

      actor_1_facebook_likes  actor_2_facebook_likes  actor_3_facebook_likes  \
0                     1000.0                   936.0                   855.0   
1                    40000.0                  5000.0                  1000.0   
2                    11000.0                   393.0                   161.0   
3                    27000.0                 23000.0                 23000.0   
4                      131.0                    12.0                     NaN   
5                      640.0                   632.0                   530.0   
6                    24000.0                 11000.0                  4000.0   
7                      799.0                   553.0                   284.0   
8                    26000.0                 21000.0                 19000.0   
9                    25000.0                 11000.0                 10000.0   
10                   15000.0                  4000.0                  2000.0   
11                   18000.0            

# Removing entry's with too many NaN values

In [29]:
def remove_too_many_NaN(data, threshold):
    print("Length before deletion: {}".format(len(data)))

    remove_indices = []
    for index, nNaN in data.isnull().sum(axis=1).iteritems():
        if nNaN >= threshold:
            remove_indices.append(index)
    
    # drop movies with too many NaNs
    data = data.drop(data.index[remove_indices])
    print("Length after deletion: {}".format(len(data)))
    
    return data


movie_data = remove_too_many_NaN(movie_data, NA_THRESH)

Length before deletion: 5043
Length after deletion: 4900


# Try testing

In [30]:
movie_num = movie_data.fillna(value=0, axis=1)
ratings = movie_num['imdb_score'].values
del movie_num['imdb_score']
#print(movie_num.head())

#ratings_integers = [int(round(rating)) for rating in ratings]

X = movie_num.values
# Data Normalization
X_std = StandardScaler().fit_transform(X)

ValueError: could not convert string to float: 'English'

Split into training set and test set. First 70% of samples is taken as training set, the remaining 30% of samples is taken as the test set

In [ ]:
print X_std.shape

# y actual values
y = movie_score

number_of_samples = len(y)

# take the first 70% of the samples as the training set
num_training_samples = int(number_of_samples*0.70)
x_train = X_std[:num_training_samples]

# take the same 70% for the y training set
y_train=y[:num_training_samples]

# take the rest as the test set
x_test = X_std[num_training_samples:]
y_test = y[num_training_samples:]

y_Train=list(y_train)

## Prediction analysis function ##

In [ ]:
def analyse_predictions(regr, x_test, y_test, doprint=False):
    predres = list(regr.predict(x_test))
    actual = list(y_test)

    highest = lowest = avg = 0
    for i in range(len(predres)):
        diff = abs(predres[i] - actual[i])
        avg += diff

        if doprint:
            print "predicted:", predres[i], 
            print "\tActual:", actual[i],
            print "\tDifference:", abs(predres[i] - actual[i])

        if diff > highest:
            highest = diff
        if diff < lowest:
            lowest = diff

    print
    print "Highest difference:", highest
    print "Lowest difference:", lowest
    print "average difference:", avg / len(predres)

# Linear Regression #

In [ ]:
from sklearn import linear_model
regr = linear_model.LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

regr.fit (x_train, y_train)
#
# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % np.mean((regr.predict(x_test) - y_test) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(x_test, y_test))

analyse_predictions(regr, x_test, y_test)


# Ridge Regression #

In [ ]:
regr = linear_model.Ridge()

regr.fit (x_train, y_train)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % np.mean((regr.predict(x_test) - y_test) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(x_test, y_test))

analyse_predictions(regr, x_test, y_test)


# Lasso #

In [ ]:
regr = linear_model.Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

regr.fit (x_train, y_train)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % np.mean((regr.predict(x_test) - y_test) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(x_test, y_test))

analyse_predictions(regr, x_test, y_test)

Print predicted values, actual values and the difference for every predicted sample

In [ ]:
regr = linear_model.BayesianRidge(alpha_1=1e-06, alpha_2=5e-06, compute_score=False, copy_X=True,
       fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
       normalize=False, tol=0.001, verbose=False)

regr.fit (x_train, y_train)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % np.mean((regr.predict(x_test) - y_test) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(x_test, y_test))

analyse_predictions(regr, x_test, y_test)